In [32]:
import pandas as pd
import json
import numpy as np
import datetime
import re
import folium
import branca.colormap as cm
from IPython.display import display
from datetime import date
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time

In [33]:
with open("japan.geojson.txt",'r+', encoding = 'utf-8') as f:
    data = f.read()
    data = re.sub(r'("nam":)\s("\w+)(\s\w+")',r'\1\2"',data)
    f.seek(0)
    f.write(data)
    f.truncate()

In [34]:
with open("japan.geojson.txt",'r+', encoding = 'utf-8') as f:
    json_obj = json.load(f)
japan_geo = json_obj

In [35]:
df = pd.read_csv('prefectures.csv')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20206 entries, 0 to 20205
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   year                         20206 non-null  int64  
 1   month                        20206 non-null  int64  
 2   date                         20206 non-null  int64  
 3   prefectureNameJ              20206 non-null  object 
 4   prefectureNameE              20206 non-null  object 
 5   testedPositive               20206 non-null  int64  
 6   peopleTested                 20083 non-null  float64
 7   hospitalized                 19657 non-null  float64
 8   serious                      17340 non-null  object 
 9   discharged                   19679 non-null  float64
 10  deaths                       19402 non-null  object 
 11  effectiveReproductionNumber  16436 non-null  float64
dtypes: float64(4), int64(4), object(4)
memory usage: 1.9+ MB


In [37]:
df.head()

,year,month,date,prefectureNameJ,prefectureNameE,testedPositive,peopleTested,hospitalized,serious,discharged,deaths,effectiveReproductionNumber
0,2020,2,8,東京都,Tokyo,3,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,2,8,神奈川県,Kanagawa,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,2,9,東京都,Tokyo,3,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,2,9,神奈川県,Kanagawa,1,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,2,10,東京都,Tokyo,3,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df['date'] = df.apply(lambda x:datetime.datetime(x.year, x.month, x.date),axis=1)

In [39]:
df.drop(['year','month','prefectureNameJ','peopleTested','hospitalized','serious','discharged','deaths','effectiveReproductionNumber'],axis=1,inplace=True)

In [40]:
df.head(20)

,date,prefectureNameE,testedPositive
0,2020-02-08,Tokyo,3
1,2020-02-08,Kanagawa,1
2,2020-02-09,Tokyo,3
3,2020-02-09,Kanagawa,1
4,2020-02-10,Tokyo,3
5,2020-02-10,Kanagawa,1
6,2020-02-11,Tokyo,3
7,2020-02-11,Kanagawa,2
8,2020-02-12,Tokyo,3
9,2020-02-12,Kanagawa,2


In [41]:
df = df.pivot_table(index='prefectureNameE',columns='date',values='testedPositive',fill_value=0).reset_index()

In [42]:
df = df.melt(id_vars="prefectureNameE", 
        var_name="days_elasped", 
        value_name="Infected")
df.head()

,prefectureNameE,days_elasped,Infected
0,Aichi,2020-02-08,0
1,Akita,2020-02-08,0
2,Aomori,2020-02-08,0
3,Chiba,2020-02-08,0
4,Ehime,2020-02-08,0


In [43]:
d0 = date(2020, 2, 8)

df['days_elasped'] = df.days_elasped.apply(lambda x: (x.date()-d0).days).astype(str)

In [44]:
grouped = df.groupby(['days_elasped','prefectureNameE']).agg({'Infected':'sum'}).reset_index(1)
grouped.loc['0',:].head(55).sort_values(by=['Infected'])

,prefectureNameE,Infected
days_elasped,,
0,Aichi,0
0,Nagano,0
0,Nagasaki,0
0,Nara,0
0,Niigata,0
0,Oita,0
0,Okayama,0
0,Okinawa,0
0,Osaka,0


In [45]:
def japan_mapper(date):
    date = str(date)
    japan_map = folium.Map(location=[36.2048, 138.2529],zoom_start=5,tiles='CartoDB positron')

    bins = list(grouped["Infected"].quantile([0, 0.50, 0.75,0.90,0.95,0.99, 1]))
    
    folium.Choropleth(
        geo_data = japan_geo,                  #json
        name ='choropleth',                  
        data = grouped.loc[date,:],                     
        columns = ['prefectureNameE', 'Infected'], #columns to work on
        key_on ='feature.properties.nam',
        fill_color = "YlOrRd",     #I passed colors Yellow,Green,Blue
        fill_opacity = 0.7,
        line_opacity = 0.2,
        bins=bins,
        reset = True
    ).add_to(japan_map)
    
    display(japan_map)

In [46]:
days= widgets.IntSlider(value=0,
    min=0,
    max=459,
    step=1,
    description='Days:')
time.sleep(0.10)
interact(japan_mapper, date = days)

interactive(children=(IntSlider(value=0, description='Days:', max=459), Output()), _dom_classes=('widget-inter…

<function __main__.japan_mapper(date)>